在上一课中，将数据管道纳入了交易算法。现在是时候定义算法如何将使用流水线生成的数据来重新平衡投资组合。我们的目标是根据尾部12个月股利与股价比率找到最大化回报的目标投资组合，同时保持由一组规则或约束条件定义的特定结构。这通常被称为投资组合优化问题。

`Optimize API`使我们能够轻松地将管道输出转化为目标和一组约束条件。 然后，我们可以使用`order_optimal_portfolio`将当前的投资组合转换为符合规范的目标投资组合。

第一步是定义一个目标。我们将使用`MaximizeAlpha`，它将尝试将资金分配给与尾部12个月股利与股价比率成比例的资产。

接下来，我们需要指定我们希望目标投资组合满足的约束列表。 我们首先在初始化时定义一些阈值并将它们存储在我们的上下文变量中：

现在，我们可以使用我们上面定义的阈值来指定`rebalance`中的约束条件：

最后，我们可以将我们的目标和约束列表传递给`order_optimal_portfolio`以计算目标投资组合，并发布将我们当前投资组合转换为最佳状态所需的订单：

# 风险管理

除了对目标投资组合的结构设置限制外，我们还希望限制可能影响其业绩的共同风险因子敞口。例如，由于stocktwits的情绪数据具有短暂性，我们打算利用情绪分数高峰期，算法可能会面临短期逆转风险。

我们将使用Quantopian的风险模型来管理我们投资组合的共同风险因子。风险模型计算资产的16种不同风险因子敞口：11个部门因子和5个主题因子（包括短期逆转）。我们可以使用`risk_loading_pipeline`函数在算法中使用这些数据，该函数返回一个数据管道，该数据管道为风险模型中的每个因子生成一列输出。

与数据管道类似，需要将风险数据管道附加到我们的算法，并提供一个名称来识别它。 然后，在`before_trading_start`中获取它的输出并将其存储在`context`中：

下一步是将`RiskModelExposure`约束添加到我们的投资组合优化逻辑。 该约束采用风险模型生成的数据，并对模型中包含的每个因子设置目标投资组合的整体风险敞口限制。

In [1]:
import pandas as pd

In [2]:
loadings = pd.DataFrame({'A':[1,2,3]}, index=['000001','000002','000003'])

In [3]:
loadings

,A
000001,1
000002,2
000003,3


In [4]:
stocks = pd.Index(['000003','000002','000001'])

In [5]:
loadings.index.get_indexer(stocks)

array([2, 1, 0])

In [6]:
loadings.A.values[loadings.index.get_indexer(stocks)]

array([3, 2, 1])